In [1]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

model = LinearRegression()
qt = QuantileTransformer(output_distribution="normal")
poly = PolynomialFeatures(degree=2)
pipe = make_pipeline(poly, model)

In [2]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
dfbase = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + dfbase
conn = create_engine(string).connect()

#### Get data

In [3]:
df = pd.read_sql(
    """
    select ticker, date, ret, roeq, bm, mom12m, mve
    from data
    where date>='2000-01'
    order by date, ticker
    """, 
    conn
)
df = df.dropna()
conn.close()

features = ["roeq", "bm", "mom12m"]
df = df.set_index(["date", "ticker"])

#### Drop largest 500 stocks each month

In [4]:
df["size_rnk"] = df.groupby("date").mve.rank(ascending=False)
df = df[df.size_rnk>500]

#### Transform features each month

In [5]:
def qt_df(d):
    x = qt.fit_transform(d)
    return pd.DataFrame(x, columns=d.columns, index=d.index)

df[features] = df.groupby("date", group_keys=False)[features].apply(qt_df)

#### Transform target each month

In [6]:
def qt_ser(s):
    x = s.copy()
    x = x.to_numpy().reshape(-1, 1)
    x = qt.fit_transform(x).flatten()
    return pd.Series(x, index=s.index)

df["target"] = df.groupby("date", group_keys=False).ret.apply(qt_ser)

#### Train and predict in a loop

In [7]:
predictions = None

dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
for train_date, end_date in zip(dates[:-1], dates[1:]):

    filter1 = df.index.get_level_values("date") < train_date
    filter2 = df.index.get_level_values("date") < end_date

    train = df[filter1]
    test = df[~filter1 & filter2]

    Xtrain = train[features]
    ytrain = train["target"]
    Xtest = test[features]

    pipe.fit(Xtrain, ytrain)
    pred = pipe.predict(Xtest)
    pred = pd.Series(pred, index=test.index)
    predictions = pd.concat((predictions, pred))

df["predict"] = predictions

#### Compute returns of portfolios of best and worst stocks

In [8]:
df = df.dropna(subset=["predict"])

numstocks = 200

df["rnk"] = df.groupby("date").predict.rank(method="first", ascending=False)
best = df[df.rnk<=numstocks]

df["rnk"] = df.groupby("date").predict.rank(method="first")
worst = df[df.rnk<=numstocks]

best_rets = best.groupby("date").ret.mean()
worst_rets = worst.groupby("date").ret.mean()
rets = pd.concat((best_rets, worst_rets), axis=1)
rets.columns = ["best", "worst"]
       
rets.to_csv("files/rets.csv")
